In [34]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder

In [35]:
train_data=pd.read_csv('train.csv')
train_data.sample(8)
target = train_data.iloc[:,-1]
train_data= train_data.iloc[:,:-1]

In [36]:
train_data.shape

(1460, 80)

In [37]:
train_data = train_data.drop("Alley",axis=1)
for i in ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
    print(train_data[i].unique())

['Gd' 'TA' 'Ex' nan 'Fa']
['TA' 'Gd' nan 'Fa' 'Po']
['No' 'Gd' 'Mn' 'Av' nan]
['GLQ' 'ALQ' 'Unf' 'Rec' 'BLQ' nan 'LwQ']
['Unf' 'BLQ' nan 'ALQ' 'Rec' 'LwQ' 'GLQ']
['SBrkr' 'FuseF' 'FuseA' 'FuseP' 'Mix' nan]
['Attchd' 'Detchd' 'BuiltIn' 'CarPort' nan 'Basment' '2Types']
['RFn' 'Unf' 'Fin' nan]
['TA' 'Fa' 'Gd' nan 'Ex' 'Po']
['TA' 'Fa' nan 'Gd' 'Po' 'Ex']


In [38]:
removed_cols = train_data.loc[:,train_data.isna().sum(axis=0)/len(train_data)*100 >= 10].columns
train_data = train_data.drop(columns=removed_cols, axis='columns')
removed_cols

Index(['LotFrontage', 'MasVnrType', 'FireplaceQu', 'PoolQC', 'Fence',
       'MiscFeature'],
      dtype='object')

In [39]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 73 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   LotShape       1460 non-null   object 
 6   LandContour    1460 non-null   object 
 7   Utilities      1460 non-null   object 
 8   LotConfig      1460 non-null   object 
 9   LandSlope      1460 non-null   object 
 10  Neighborhood   1460 non-null   object 
 11  Condition1     1460 non-null   object 
 12  Condition2     1460 non-null   object 
 13  BldgType       1460 non-null   object 
 14  HouseStyle     1460 non-null   object 
 15  OverallQual    1460 non-null   int64  
 16  OverallCond    1460 non-null   int64  
 17  YearBuilt      1460 non-null   int64  
 18  YearRemo

In [40]:
columns_with_null=train_data.loc[:,(train_data.isna().sum(axis=0)>0)].columns

In [41]:
columns_with_null

Index(['MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinType2', 'Electrical', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageQual', 'GarageCond'],
      dtype='object')

In [42]:
train_data["BsmtCond"].dtype

dtype('O')

In [43]:
# l1 list contain columns with Object datatype
# l2 list contain columns with numerical datatype
l1,l2=[],[]

In [44]:
for col in columns_with_null:
    if train_data[col].dtype == 'O':
        l1.append(col)
    else:
        l2.append(col)


In [45]:
print(l1)
print(l2)

['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']
['MasVnrArea', 'GarageYrBlt']


In [46]:
for i in l1:
    print(train_data[i].unique())

['Gd' 'TA' 'Ex' nan 'Fa']
['TA' 'Gd' nan 'Fa' 'Po']
['No' 'Gd' 'Mn' 'Av' nan]
['GLQ' 'ALQ' 'Unf' 'Rec' 'BLQ' nan 'LwQ']
['Unf' 'BLQ' nan 'ALQ' 'Rec' 'LwQ' 'GLQ']
['SBrkr' 'FuseF' 'FuseA' 'FuseP' 'Mix' nan]
['Attchd' 'Detchd' 'BuiltIn' 'CarPort' nan 'Basment' '2Types']
['RFn' 'Unf' 'Fin' nan]
['TA' 'Fa' 'Gd' nan 'Ex' 'Po']
['TA' 'Fa' nan 'Gd' 'Po' 'Ex']


In [47]:
train_data.loc[:,l1].dtypes

BsmtQual        object
BsmtCond        object
BsmtExposure    object
BsmtFinType1    object
BsmtFinType2    object
Electrical      object
GarageType      object
GarageFinish    object
GarageQual      object
GarageCond      object
dtype: object

In [48]:
train_data.loc[:,l2].dtypes

MasVnrArea     float64
GarageYrBlt    float64
dtype: object

In [49]:
train_data['BsmtQual'].value_counts()

BsmtQual
TA    649
Gd    618
Ex    121
Fa     35
Name: count, dtype: int64

In [50]:
np.sort(train_data.columns)
print(l2,l1)

['MasVnrArea', 'GarageYrBlt'] ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']


In [51]:
# l3 list contain ordinal columns
# Complete this l3 list and only write map for the columns' name given above, don't touch anything if you don;t know about that and don't directly merge your branch into master
l3 = ['MSZoning','Street','LotShape','LandContour','Utilities','LotConfig','LandSlope','BldgType','HouseStyle','RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd','ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond','BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC','PavedDrive' ,'GarageCond', 'GarageQual','KitchenQual', 'Functional', 'GarageType','GarageFinish']
maps = [
        
	    ['A','C (all)','FV','I','RH','RL','RP','RM'],
        ['Grvl','Pave'],
        ['IR3', 'IR2', 'IR1', 'Reg'],
        ['Low', 'HLS', 'Bnk', 'Lvl'],
        ['ELO', 'NoSeWa', 'NoSewr', 'AllPub'],
        ['FR3', 'FR2', 'CulDSac', 'Corner', 'Inside'],
        ['Sev', 'Mod', 'Gtl'],
        ['1Fam','2fmCon','Duplex','TwnhsE','Twnhs'],
        ['1Story','1.5Fin','1.5Unf','2Story','2.5Fin','2.5Unf','SFoyer','SLvl'],
	    ['Mansard', 'Gambrel', 'Flat', 'Shed', 'Gable', 'Hip'],
    	['Roll', 'Membran', 'Tar&Grv', 'CompShg', 'WdShngl', 'WdShake', 'Metal', 'ClyTile'],
    	['ImStucc', 'CBlock', 'BrkComm', 'AsphShn', 'AsbShng', 'WdShing', 'Wd Sdng', 'MetalSd', 'Plywood', 'HdBoard', 'VinylSd', 'Stucco', 'CemntBd', 'BrkFace', 'Stone'],
    	['Other', 'ImStucc', 'CBlock', 'Brk Cmn', 'AsphShn', 'AsbShng', 'Wd Shng', 'Wd Sdng', 'MetalSd', 'Plywood', 'HdBoard', 'VinylSd', 'Stucco', 'CmentBd', 'BrkFace', 'Stone'],
       	['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    	['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    	['Stone', 'Slab', 'CBlock', 'BrkTil', 'Wood', 'PConc'],
    	['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    	['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    	['No', 'Mn', 'Av', 'Gd'],
    	['Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
    	['Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
    	["NA","Po","Fa","TA","Gd","Ex"],
	    ['N', 'P','Y'],
        ["NA","Po","Fa","TA","Gd","Ex"],
        ["NA","Po","Fa","TA","Gd","Ex"],
     	 ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    ['Sal', 'Sev', 'Maj2', 'Maj1', 'Mod', 'Min2', 'Min1', 'Typ'],
    ['NA', 'Detchd', 'CarPort', 'BuiltIn', 'Basment', 'Attchd', '2Types'],
    ['NA', 'Unf', 'RFn', 'Fin'] 
    ]

In [52]:
len(maps), len(l3)

(29, 29)

In [53]:
for i in l3:
    print(i, train_data[i].unique())
train_data['MSZoning'].value_counts()

MSZoning ['RL' 'RM' 'C (all)' 'FV' 'RH']
Street ['Pave' 'Grvl']
LotShape ['Reg' 'IR1' 'IR2' 'IR3']
LandContour ['Lvl' 'Bnk' 'Low' 'HLS']
Utilities ['AllPub' 'NoSeWa']
LotConfig ['Inside' 'FR2' 'Corner' 'CulDSac' 'FR3']
LandSlope ['Gtl' 'Mod' 'Sev']
BldgType ['1Fam' '2fmCon' 'Duplex' 'TwnhsE' 'Twnhs']
HouseStyle ['2Story' '1Story' '1.5Fin' '1.5Unf' 'SFoyer' 'SLvl' '2.5Unf' '2.5Fin']
RoofStyle ['Gable' 'Hip' 'Gambrel' 'Mansard' 'Flat' 'Shed']
RoofMatl ['CompShg' 'WdShngl' 'Metal' 'WdShake' 'Membran' 'Tar&Grv' 'Roll'
 'ClyTile']
Exterior1st ['VinylSd' 'MetalSd' 'Wd Sdng' 'HdBoard' 'BrkFace' 'WdShing' 'CemntBd'
 'Plywood' 'AsbShng' 'Stucco' 'BrkComm' 'AsphShn' 'Stone' 'ImStucc'
 'CBlock']
Exterior2nd ['VinylSd' 'MetalSd' 'Wd Shng' 'HdBoard' 'Plywood' 'Wd Sdng' 'CmentBd'
 'BrkFace' 'Stucco' 'AsbShng' 'Brk Cmn' 'ImStucc' 'AsphShn' 'Stone'
 'Other' 'CBlock']
ExterQual ['Gd' 'TA' 'Ex' 'Fa']
ExterCond ['TA' 'Gd' 'Fa' 'Po' 'Ex']
Foundation ['PConc' 'CBlock' 'BrkTil' 'Wood' 'Slab' 'Stone']
BsmtQu

MSZoning
RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: count, dtype: int64

In [54]:
# l4 list contain nominal columns
# l5 list contains numerical columns
l4 = []
l5 = []
for col in train_data.columns:
    if train_data[col].dtype == 'O':
        if col not in l3:
            l4.append(col)
    else:
        l5.append(col)

In [55]:
# impute then encode
# transformer = ColumnTransformer(transformers=[
#     ('tnf1',SimpleImputer(strategy="mean"),l2),
#     ('tnf2',SimpleImputer(strategy="most_frequent"),l1),
# ],remainder='passthrough')
# train_data

In [56]:
# transformed_train_data = pd.DataFrame(transformer.fit_transform(train_data),columns=train_data.columns)


In [57]:
# transformed_train_data

In [58]:
train_data.isna().sum().sum()
l3
train_data['MSZoning'].unique()

array(['RL', 'RM', 'C (all)', 'FV', 'RH'], dtype=object)

In [59]:
for i in ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
    print(train_data[i].unique())

['Gd' 'TA' 'Ex' nan 'Fa']
['TA' 'Gd' nan 'Fa' 'Po']
['No' 'Gd' 'Mn' 'Av' nan]
['GLQ' 'ALQ' 'Unf' 'Rec' 'BLQ' nan 'LwQ']
['Unf' 'BLQ' nan 'ALQ' 'Rec' 'LwQ' 'GLQ']
['SBrkr' 'FuseF' 'FuseA' 'FuseP' 'Mix' nan]
['Attchd' 'Detchd' 'BuiltIn' 'CarPort' nan 'Basment' '2Types']
['RFn' 'Unf' 'Fin' nan]
['TA' 'Fa' 'Gd' nan 'Ex' 'Po']
['TA' 'Fa' nan 'Gd' 'Po' 'Ex']


In [60]:
for i in l2:
    train_data[i].fillna(train_data[i].mean(),inplace=True)
for i in l1:
    train_data[i].fillna(train_data[i].mode().values[0],inplace=True)
    # print(train_data[i].unique())
    # print(type(train_data.loc[0:i]))

/tmp/ipykernel_5760/107827536.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data[i].fillna(train_data[i].mean(),inplace=True)
/tmp/ipykernel_5760/107827536.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)',

In [61]:
print(train_data['BsmtQual'].mode().values[0])

TA


In [62]:
transformer2 = ColumnTransformer([
    ('tnf1',OrdinalEncoder(categories=maps),l3),
    ('tnf2',OneHotEncoder(sparse_output=False,drop='first'),l4),
],remainder='passthrough')

In [63]:
m = 0
for i in l3:
    print(train_data[i].value_counts(),"    ", maps[m])
    m+=1

MSZoning
RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: count, dtype: int64      ['A', 'C (all)', 'FV', 'I', 'RH', 'RL', 'RP', 'RM']
Street
Pave    1454
Grvl       6
Name: count, dtype: int64      ['Grvl', 'Pave']
LotShape
Reg    925
IR1    484
IR2     41
IR3     10
Name: count, dtype: int64      ['IR3', 'IR2', 'IR1', 'Reg']
LandContour
Lvl    1311
Bnk      63
HLS      50
Low      36
Name: count, dtype: int64      ['Low', 'HLS', 'Bnk', 'Lvl']
Utilities
AllPub    1459
NoSeWa       1
Name: count, dtype: int64      ['ELO', 'NoSeWa', 'NoSewr', 'AllPub']
LotConfig
Inside     1052
Corner      263
CulDSac      94
FR2          47
FR3           4
Name: count, dtype: int64      ['FR3', 'FR2', 'CulDSac', 'Corner', 'Inside']
LandSlope
Gtl    1382
Mod      65
Sev      13
Name: count, dtype: int64      ['Sev', 'Mod', 'Gtl']
BldgType
1Fam      1220
TwnhsE     114
Duplex      52
Twnhs       43
2fmCon      31
Name: count, dtype: int64      ['1Fam', '2fmCon', 'Dupl

In [64]:
new_data = transformer2.fit_transform(train_data) 


In [65]:
new_data

array([[5.000e+00, 1.000e+00, 3.000e+00, ..., 0.000e+00, 2.000e+00,
        2.008e+03],
       [5.000e+00, 1.000e+00, 3.000e+00, ..., 0.000e+00, 5.000e+00,
        2.007e+03],
       [5.000e+00, 1.000e+00, 2.000e+00, ..., 0.000e+00, 9.000e+00,
        2.008e+03],
       ...,
       [5.000e+00, 1.000e+00, 3.000e+00, ..., 2.500e+03, 5.000e+00,
        2.010e+03],
       [5.000e+00, 1.000e+00, 3.000e+00, ..., 0.000e+00, 4.000e+00,
        2.010e+03],
       [5.000e+00, 1.000e+00, 3.000e+00, ..., 0.000e+00, 6.000e+00,
        2.008e+03]])

In [66]:
len(new_data[0])

127

In [67]:
from sklearn.linear_model import LinearRegression

In [68]:
lr = LinearRegression()
model = lr.fit(new_data,target)

In [69]:
test = pd.read_csv('test.csv')

In [70]:
test = test.drop("Alley",axis=1)

In [71]:
test = test.drop(columns=removed_cols, axis='columns')

In [72]:
columns_with_null=test.loc[:,(test.isna().sum(axis=0)>0)].columns
for col in columns_with_null:
    if train_data[col].dtype == 'O':
        l1.append(col)
    else:
        l2.append(col)
        
for i in l2:
    test[i].fillna(test[i].mean(),inplace=True)
for i in l1:
    test[i].fillna(test[i].mode().values[0],inplace=True)
test.isna().sum().sum()

/tmp/ipykernel_5760/1724824472.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test[i].fillna(test[i].mean(),inplace=True)
/tmp/ipykernel_5760/1724824472.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin

0

In [73]:
test_transformed = transformer2.transform(test)

In [74]:
len(test_transformed[0])

127

In [75]:
pred = model.predict(test_transformed)

In [81]:
submission = pd.DataFrame({'Id':test['Id'], 'SalePrice': pred}).reset_index().iloc[:,1:].set_index('Id')

In [82]:
submission.to_csv('submission.csv')